In [5]:
import requests
import twint
import pandas as pd

custom_query = ""
mentioned = {}
replied = {}

mentions = ['This Love Dani Atkins', 'Love Song Sophia Bennett', 'Letters to the Lost Iona Grey']
replies = ['This Love Dani Atkins', 'Love Song Sophia Bennett', 'Letters to the Lost Iona Grey']

for m in mentions:
    mentioned.update({m: {}})
    custom_query += "@{} OR ".format(m)
custom_query = custom_query[:-3] # -3 because we want to leave a space

for r in replies:
    replied.update({r: {}})
    custom_query += "to:{} OR ".format(r)
custom_query = custom_query[:-4]

# Twint setup here
c = twint.Config()
c.Custom_query = custom_query
c.Store_object = True
c.Store_csv = True
c.Output = "tweets_mentions_replies.csv"

# we want to hide the output, there will be a lot of tweets and the terminal might crash
c.Hide_output = True

twint.run.Search(c)

tweets = twint.output.tweets_list

# now iterate over the tweets to do a bit of statistics
# we will determine the most mentioned users
# and the user that got the most replies

RuntimeError: This event loop is already running

In [3]:
for t in tweets:
    # iterate over the mentioned users
    for m in t.mentions:
        try:
            mentioned[m]['count'] += 1
        except ValueError:
            mentioned.update({m: {'by': t.username, 'count': 1}})

    # we don't have the user which we reply in the tweet object, tweet
    # but from the tweet ID we can get redirected to the original URL
    # and from the URL, extract the username

    _reply_to = requests.get('https://twitter.com/statuses/{}'.format(t.conversation_id)).request.path_url.split('/')[1]
    try:
        replied[_reply_to]['count'] += 1
    except KeyError:
        replied.update({_reply_to: {'by': t.username, 'count': 1}})
    print('.', end='', flush=True)

# and now save to CSV for further analysis
actions = {'mentioned': mentioned, 'replied': replied}
for a in actions:
    action = actions[a]
    with open('{}.csv'.format(a), 'w') as output:
        output.write('author,{},count\n'.format(a))
        for user in action:
            output.write('{},{},{}\n'.format(action[user]['by'], user, action[user]['count']))

NameError: name 'tweets' is not defined